# Email Data Pipeline

This notebook extracts data from qardio google sheets, cleans the data, and loads it into the Qardio SQL database

In [3]:
from google_apis import gsheets_functions as gs
import pandas as pd
import sqlalchemy

database = 'qardio'
host = '127.0.0.1'
user = 'root'
password = 'Party100'
url = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = sqlalchemy.create_engine(url, echo=True)
conn = engine.connect()

2023-06-21 19:50:38,496 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-06-21 19:50:38,498 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-21 19:50:38,503 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-06-21 19:50:38,505 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-21 19:50:38,507 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-06-21 19:50:38,510 INFO sqlalchemy.engine.Engine [raw sql] {}


In [5]:
sales_data = gs.gspread_read(wb='1dKiIp2ETyzfKHzR9fMeHrgf2N_UKcSa5uPWathJzgIw',
                              ws='Sales')

In [7]:
sales_data.columns = sales_data.columns.str.lower()
sales_data.columns = sales_data.columns.str.strip()
sales_data.columns = sales_data.columns.str.replace(' ', '_')

In [8]:
sales_data = sales_data.set_index('sale_id')
sales_data.start_date = pd.to_datetime(sales_data.start_date)
sales_data.end_date = pd.to_datetime(sales_data.end_date)

## Save data to SQL database

In [9]:
sales_data.to_sql(name='sales',
          con=conn,
          if_exists='replace')

2023-06-21 19:53:54,156 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-21 19:53:54,168 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-06-21 19:53:54,182 INFO sqlalchemy.engine.Engine [generated in 0.01437s] {'table_schema': 'qardio', 'table_name': 'sales'}
2023-06-21 19:53:54,202 INFO sqlalchemy.engine.Engine 
CREATE TABLE sales (
	sale_id BIGINT, 
	sale_amount BIGINT, 
	start_date DATETIME, 
	end_date DATETIME, 
	`duration_(days)` BIGINT
)


2023-06-21 19:53:54,203 INFO sqlalchemy.engine.Engine [no key 0.00155s] {}
2023-06-21 19:53:54,224 INFO sqlalchemy.engine.Engine CREATE INDEX ix_sales_sale_id ON sales (sale_id)
2023-06-21 19:53:54,225 INFO sqlalchemy.engine.Engine [no key 0.00148s] {}
2023-06-21 19:53:54,249 INFO sqlalchemy.engine.Engine INSERT INTO sales (sale_id, sale_amount, start_date, end_date, `duration_(days)`) VALUES (%(sale_id)s, %(sale_amount)s, %(start_date)

16